Final Project:
Naomi Provost, Kenny Ngo, David Pontecorvo, Gabe Reinchenberger, Najmi Salim

In [1]:
import sys #import sys so I can append the path where PyGreSQL
sys.path.append('C:\Python27\ArcGIS10.3\Lib\site-packages')
import pg #import PyGreSQL
from pg import DB #import the DB class so we can create a connection
import csv #import csv to use csv reader

In [2]:
db = DB(dbname = 'geog465_najmi93', host = 'geog-db2.geog.uw.edu', port = 5432, user = 'najmi93', passwd = 'najmi93')

db.query('create table location (id int PRIMARY KEY, address varchar(200), latitude varchar(30), longitude varchar(30))')

db.query('''create table application_information(id int PRIMARY KEY, application_permit int, applicant_name varchar(100), application_date varchar(20), 
         application_status varchar(100), location_id int , project_information_id int)''')

db.query('''create table project_information(id int PRIMARY KEY, application_permit int, 
    action_type varchar(200), work_type varchar(200), value varchar(30), location_id int)''')

db.query('''create table building_permits (id int PRIMARY KEY, application_permit int, address varchar(200),
    location_id int, project_information_id int, application_information_id int)''')

db.query('create table permit_information(id int PRIMARY KEY, application_permit int,\
permit_type varchar(200),\
description varchar(800),\
category varchar(200),\
contractor varchar(100),\
master_use_permit varchar(20),\
permit_issue_date varchar(20),\
final_date varchar(20),\
expiration_date varchar(20), location_id int, project_information_id int, application_information_id int)')

def write_list(filename):
    #takes in filename and returns list of information in file
    f = open(filename)
    reader = csv.reader(f, delimiter = ',')
    my_list = []
    next(reader, None)
    for row in reader:
        my_list.append(row)
    f.close()
    return my_list

#creates lists for each table
application_list = write_list('C:/Users/ninip/Desktop/5_25/application_information.csv')
building_list = write_list('C:/Users/ninip/Desktop/5_25/building_information_permits.csv')
permit_list = write_list('C:/Users/ninip/Desktop/5_25/permit_information.csv')
project_list = write_list('C:/Users/ninip/Desktop/5_25/project_information.csv')
location_list = write_list('C:/Users/ninip/Desktop/5_25/location.csv')

#populate tables
db.inserttable('permit_information', permit_list)
db.inserttable('building_permits', building_list)
db.inserttable('application_information', application_list)
db.inserttable('location', location_list)
db.inserttable('project_information', project_list)

#connect application information id to location id using building_permits as a link table
db.query('''update application_information set location_id = location.id from location, building_permits
where building_permits.application_information_id = application_information.id and
building_permits.address = location.address''')

#connect application information id to project_information id using building_permits as a link table
db.query('''update application_information set project_information_id = project_information.id from project_information, building_permits
where building_permits.application_information_id = application_information.id and
building_permits.application_permit = project_information.application_permit''')

#connect project_information information id to location id using building_permits as a link table
db.query('''update project_information set location_id = location.id from location, building_permits
where building_permits.project_information_id = project_information.id and
building_permits.address = location.address''')

#connect permit_information id to location id using building_permits as a link table
db.query('''update permit_information set location_id = location.id from location, building_permits
where building_permits.id = permit_information.id and
building_permits.address = location.address''')

#connect permit information id to project_information id using building_permits as a link table
db.query('''update permit_information set project_information_id = project_information.id from project_information, building_permits
where building_permits.id = permit_information.id and
building_permits.application_permit = project_information.application_permit''')

#connect permit information id to application_information id using building_permits as a link table
db.query('''update permit_information set application_information_id = application_information.id from application_information, building_permits
where building_permits.id = permit_information.id and
building_permits.application_information_id = application_information.id''')

In [7]:
location_query = db.query('select * from location')
application_query = db.query('select * from application_information')
permit_query = db.query('select * from permit_information')
building_query = db.query('select * from building_permits')
project_query = db.query('select * from project_information')

In [11]:
def write_csv(output, columnlist, fun):
    f = output
    writer = csv.DictWriter(f, fieldnames = columnlist)
    writer.writeheader()
    writer.writerows(fun.dictresult())
    f.close()

In [ ]:
project_info_output = open("C:/Users/ninip/Desktop/project_information_output.csv", 'w') #opens output file
permit_info_output = open("C:/Users/ninip/Desktop/permit_information_output.csv", 'w') #opens output file
application_info_output = open("C:/Users/ninip/Desktop/application_information_output.csv", 'w') #opens output file
location_output = open("C:/Users/ninip/Desktop/location_output.csv", 'w') #opens output file
building_output = open("C:/Users/ninip/Desktop/building_information_permits_output.csv", 'w') #opens output file

project_info_fieldnames = ['id','application_permit', 'action_type', 'work_type', 'value', 'location_id']
permit_info_fieldnames = ['id','application_permit','permit_type', 'description', 'category', 'contractor', 'master_use_permit', 'permit_issue_date', 'final_date', 'expiration_date',\
                          'location_id', 'project_information_id', 'application_information_id']
application_info_fieldnames = ['id', 'application_permit', 'applicant_name', 'application_date', 'application_status', 'location_id', 'project_information_id']
location_fieldnames = ['id', 'address', 'latitude', 'longitude']
building_fieldnames = ['id', 'application_permit', 'address', 'location_id', 'project_information_id', 'application_information_id']

write_csv(project_info_output, project_info_fieldnames, project_query)
write_csv(permit_info_output, permit_info_fieldnames, permit_query)
write_csv(application_info_output, application_info_fieldnames, application_query)
write_csv(location_output, location_fieldnames, location_query)
write_csv(building_output, building_fieldnames, building_query)